In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import math
import numpy as np
import pandas as pd
import pyomo.environ as opt

In [ ]:
pd.options.plotting.backend = "plotly"
template = "plotly_dark"

In [ ]:
from pyomo.core.base.param import ScalarParam, IndexedParam

In [ ]:
from datetime import datetime, timedelta

In [ ]:
from optses.coupling import Load, GridSlack
from optses.storage.ecm import RintModel
from optses.storage.converter import QuadraticLossConverter
from optses.storage.system import StorageSystem

# Model

In [ ]:
profile = pd.read_csv("data/power_profile.csv", date_format="%Y-%m-%d %H:%M:%S%z", index_col=0)
profile = profile["ac_power_p_poi_sim"] / 10
profile.index = profile.index.tz_localize(None) # drop timezone

In [ ]:
class OptModel:
    def __init__(self, profile):
        # super().__init__(solver=solver)
        self.model = opt.ConcreteModel()
        self.solver=opt.SolverFactory("ipopt")
        self.build_model(profile)

    def build_model(self, profile):
        # opt model
        model = self.model

        # time parameters
        dt = (profile.index[1] - profile.index[0]).seconds / 3600
        timesteps = len(profile)
        model.time = opt.RangeSet(0, timesteps - 1)
        model.dt =   opt.Param(initialize=dt)

        ## cell
        # nominal values
        system_energy = 30e3 # 30 kWh
        system_power = 30e3 # 15 kW
        system_voltage = 400 # V
        cell_capacity = 3.0 # Ah
        cell_voltage = 3.2 # V

        # required cells in parallel / serial
        s = math.ceil(system_voltage / cell_voltage)  # parallel cells
        p = math.ceil(system_energy / (s * cell_voltage) / cell_capacity)
        circuit = dict(p=p, s=s)

        ## ocv function 
        # parameters
        ocvp = ( 
            2.900074932012732,
            4.927554283419728,
            -29.299690476948975,
            95.9761469315301,
            -181.93721324798557,
            199.7074219198455,
            -117.75695789380974,
            28.844629090305514,
        )

        # pyomo constraint function
        def ocv(b, t):
            return b.ocv[t] == (
                ocvp[0]
                + ocvp[1] * b.soc[t]
                + ocvp[2] * b.soc[t] ** 2
                + ocvp[3] * b.soc[t] ** 3
                + ocvp[4] * b.soc[t] ** 4
                + ocvp[5] * b.soc[t] ** 5
                + ocvp[6] * b.soc[t] ** 6
                + ocvp[7] * b.soc[t] ** 7
            )

        cell_model = RintModel(
            capacity=3.0, r0=0.045, ocv=ocv, circuit=circuit,
            soc_start=0.5, soc_bounds=(0.1, 0.95), i_bounds=(3.0, 6.0)
        )

        # converter
        k = [0.00650714, 0.00192592, 0.03156912,]
        converter_model = QuadraticLossConverter(system_power, k[0], k[1], k[2])

        # system
        sys = StorageSystem(cell_model, converter_model)

        for i in (1, 2):
            model.add_component(name=f"system{i}", val=opt.Block(rule=sys.build))


        # 
        load = Load(profile)
        model.setpoint = opt.Block(rule=load.build)

        slack = GridSlack(penalty=1e9)
        model.slack = opt.Block(rule=slack.build)

        @model.Constraint(model.time)
        def power_balance(m, t):
            return m.system1.power[t] + m.system2.power[t] == m.setpoint.power[t] + m.slack.power[t]
        
        @model.Objective()
        def objective(m):
            cell_losses = sum(m.system1.cell_loss_dc[t] + m.system2.cell_loss_dc[t] for t in model.time)
            converter_losses = sum(m.system1.converter_loss[t] + m.system2.converter_loss[t] for t in model.time)
            return cell_losses + converter_losses + m.slack.penalty_cost
    
    def update_model(self, val_dict: dict) -> None:
        model = self.model

        for block_name, block_values in val_dict.items():
            block = model.find_component(block_name)

            for param_name, val in block_values.items():
                param = block.find_component(param_name)
                if isinstance(param, ScalarParam):
                    param.set_value(val)
                elif isinstance(param, IndexedParam):
                    for t in model.time:
                        param[t].set_value(val.iloc[t])

    def recover_results(self):
        m = self.model
        return pd.DataFrame({
            # "power_cell": np.array([opt.value(m.system1.cell_power[t]) for t in m.time]),
            # "power_dc": np.array([opt.value(m.system1.power_dc[t]) for t in m.time]),
            # "power_ac": np.array([opt.value(m.system1.power[t]) for t in m.time]),
            # "i1": np.array([opt.value(m.system1.i[t]) for t in m.time]),
            # "ic1": np.array([opt.value(m.system1.ic[t]) for t in m.time]),
            # "id1": np.array([opt.value(m.system1.id[t]) for t in m.time]),
            # "idc1": np.array([opt.value(m.system1.i_dc[t]) for t in m.time]),
            "power1": np.array([opt.value(m.system1.power[t]) for t in m.time]),
            "power2": np.array([opt.value(m.system2.power[t]) for t in m.time]),
            "soc1": np.array([opt.value(m.system1.soc[t]) for t in m.time]),
            "soc2": np.array([opt.value(m.system2.soc[t]) for t in m.time]),
            "slack": np.array([opt.value(m.slack.power[t]) for t in m.time]),
            "setpoint": np.array([opt.value(m.setpoint.power[t]) for t in m.time]),
            # "cell_loss": np.array([opt.value(m.system1.cell_loss_dc[t]) for t in m.time]),
            # "converter_loss": np.array([opt.value(m.system1.converter_loss[t]) for t in m.time]),
        })

    def solve(self, *updates):
        if updates:
            for update in updates:
                self.update_model(update)
        
        status = self.solver.solve(self.model)

        if status["Solver"][0]["Termination condition"] == "optimal":
            return self.recover_results()
        else:
            return None # TODO: best way to handle?
        

In [ ]:
power_split = OptModel(profile.loc["2022-07-01 00:00":"2022-07-01 23:45"])
# res = power_split.solve()

In [ ]:
params = {
    "system1": {
        "soc_start": 0.6,
        "r0": 0.45
    },
    "setpoint": {
        "power": profile.loc["2022-07-02 00:00":"2022-07-02 23:45"]
    }
}
power_split.update_model(params)
res = power_split.solve()

In [ ]:
res = power_split.solve(params)
res

# MPC

In [ ]:
dt = 900 # seconds = 15 min
timestep = timedelta(seconds=dt)
horizon = timedelta(days=1, seconds=-dt)

timesteps = pd.date_range(
    start= "2022-07-01 00:00:00", 
    end  = "2022-07-03 00:00:00", 
    freq = timedelta(days=1)
)

In [ ]:
df_mpc = pd.DataFrame()
soc1 = 0.5
soc2 = 0.6

for time in timesteps:
    timerange = pd.date_range(start=time, end=time+horizon, freq=timestep)

    params = {
        "system1": {
            "soc_start": soc1,
        },
        "system2":{
            "soc_start": soc2,
        },
        "setpoint": {
            "power": profile.loc[timerange]
        },
    }
    attempts = 0
    while attempts < 3:
        res = power_split.solve(params)
        if res is not None:
            break
        attempts += 1
        print(attempts)

    res.index = timerange
    df_mpc = pd.concat([df_mpc, res])

In [ ]:
df_mpc[["soc1", "soc2"]].plot(template=template)